In [2]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v3/codes


In [3]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v3


In [4]:
from codes.model import my_model_for_toto as my_model
import numpy as np
import pandas as pd
import pickle

def get_toto_sim(toto_n, toto_kind):

    if toto_kind == 'toto':
        f_name = 'toto_dic.sav'
    elif toto_kind == 'mini toto-A組':
        f_name = 'toto_miniA_dic.sav'
    else:
        f_name = 'toto_miniB_dic.sav'

    try:
        toto_dic = pickle.load(open('data/simulation_toto/' + f_name, 'rb'))
    except:
        toto_dic = {}
    
    if toto_n in toto_dic:
        toto_sim = toto_dic[toto_n]
    else:
        toto_sim = toto_simulator(toto_n, toto_kind)
        toto_dic[toto_n] = toto_sim
        pickle.dump(toto_dic, open('data/simulation_toto/' + f_name, 'wb'))
    return toto_sim
    
class toto_simulator():
    
    def __init__(self, toto_n, toto_kind):
        self.toto_n = toto_n
        self.toto_kind = toto_kind
        self.df_pred = my_model.pred_toto(self.toto_n, self.toto_kind)
        
    def show_df(self):
        df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)

        df_toto = df_toto[['第n回', '開催日', '種別', 'ホーム', 'アウェイ', 'くじ結果']]
        df_toto = df_toto[(df_toto['種別'] == self.toto_kind) & (df_toto['第n回'] == self.toto_n)]

        df_result = pd.merge(df_toto, self.df_pred, how = 'left')
        
        columns = ['第n回', '開催日', '種別', 'ホーム', 'アウェイ','分', '勝', '敗', 'くじ結果']
        df_result = df_result.reindex(columns=columns)
        return df_result.rename(columns={'勝': '勝(1)', '敗': '敗(2)', '分': '分(0)'})
        
    def get_buy_list(self, toto_funds):
        
        pred_list = np.array(self.df_pred[['分', '勝', '敗']].values)
        buy_list = np.array(np.zeros([len(pred_list),3]))
    
        # 各試合で予測確率が最大のものを購入
        for i in range(len(pred_list)):
            max_idx = np.argmax(pred_list[i])
            pred_list[i][max_idx] = 0
            buy_list[i][max_idx] = 1
        purchase_price, _, _, _ = self.calc_purchase_price(buy_list.copy())

        continue_buy_flg = True
        while continue_buy_flg:
            # 新たに購入
            new_pred_list, new_buy_list = self.buy_one_more(pred_list, buy_list)
            
            # 購入金額を算出
            new_purchase_price, single, double, triple = self.calc_purchase_price(new_buy_list)
            
            # さらに購入できるかをチェック
            continue_buy_flg = self.check_continue(new_purchase_price, single, double, triple, toto_funds)
            
            if continue_buy_flg:
                pred_list = new_pred_list.copy()
                buy_list = new_buy_list.copy()
                purchase_price = new_purchase_price
                
        df_buy = pd.concat([self.df_pred.copy(), pd.DataFrame(buy_list)], axis = 1)
        df_buy.drop(columns = ['勝', '敗', '分'], inplace = True)
        
        return df_buy, purchase_price
                
    def buy_one_more(self, pred_list, buy_list):
        pred_list, buy_list = pred_list.copy(), buy_list.copy()
        max_val = pred_list.max()
        for i in range(len(pred_list)):
            if max_val == pred_list[i].max():
                max_idx = np.argmax(pred_list[i])
                pred_list[i][max_idx] = 0
                buy_list[i][max_idx] = 1
        return pred_list, buy_list
    
    def check_continue(self, purchase_price, single, double, triple, toto_funds):
        if toto_funds < purchase_price:
            return False
        
        if self.toto_kind[:4] == 'toto':
            if double == 0:
                if 5 < triple:
                    return False
            elif double == 1:
                if 4 < triple:
                    return False
            elif double == 2:
                if 4 < triple:
                    return False
            elif double == 3:
                if 3 < triple:
                    return False
            elif double == 4:
                if 3 < triple:
                    return False
            elif double == 5:
                if 2 < triple:
                    return False
            elif double == 6:
                if 1 < triple:
                    return False
            elif double == 7:
                if 1 < triple:
                    return False
            elif double == 8:
                if 0 < triple:
                    return False
            else:
                return False
        else:
             if 5 == triple:
                return False
        return True
    
    def calc_purchase_price(self, buy_list):
        single = 0
        double = 0
        triple = 0
        for row in buy_list:
            count_buy = np.sum(row)

            if count_buy == 1:
                single+=1
            elif count_buy == 2:
                double +=1
            else:
                triple += 1
        purchase_price = 100*(2**double)*(3**triple)

        return purchase_price, single, double, triple
    
    def get_result(self, price):
        df_buy, purchase_price = self.get_buy_list(price)
        df_buy['種別'] = self.toto_kind

        df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)

        df_toto = df_toto[['第n回', '開催日', '種別', 'ホーム', 'アウェイ', 'くじ結果']]
        df_toto = df_toto[(df_toto['種別'] == self.toto_kind) & (df_toto['第n回'] == self.toto_n)]

        df_result = pd.merge(df_toto, df_buy, how = 'right')
        
        if df_result[:1]['くじ結果'].values[0] == '未実施':
            df_result[0] = df_result[0].astype(int)
            df_result[1] = df_result[1].astype(int)
            df_result[2] = df_result[2].astype(int)
            
            columns = ['第n回', '開催日', '種別', 'ホーム', 'アウェイ', 'くじ結果', 1,0,2]
            df_result = df_result.reindex(columns=columns)
            return df_result
        
        df_result['くじ結果'] = df_result['くじ結果'].astype(int)

        winning_list = []
        for i, row in df_result.iterrows():
            idx = row['くじ結果']
            if (idx == 0) | (idx == 1) | (idx == 2):
                if row[idx] == 1:
                    winning_list += [1]
                else:
                    winning_list += [0]
            else:
                winning_list += [1]
        df_result = df_result.assign(当選フラグ = winning_list)

        count_miss = sum(df_result['当選フラグ'].values == 0)
        df_result['外したくじ数'] = count_miss
        
        df_result[0] = df_result[0].astype(int)
        df_result[1] = df_result[1].astype(int)
        df_result[2] = df_result[2].astype(int)

        return df_result

In [215]:
df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)
df_toto = df_toto[df_toto['開催日'] < 20220000].sort_values('開催日', ascending = False)
df_toto = df_toto[df_toto['開催日'] > 20210000].sort_values('開催日', ascending = False)
df_toto = df_toto[['第n回', '種別']].drop_duplicates().reset_index(drop = True)
df_toto = df_toto[df_toto['第n回'] < 1240]

df_toto_toto = df_toto[df_toto['種別']=='toto'].reset_index(drop = True)
df_toto_miniA = df_toto[df_toto['種別']=='mini toto-A組'].reset_index(drop = True)
df_toto_miniB = df_toto[df_toto['種別']=='mini toto-B組'].reset_index(drop = True)

In [216]:
toto_dic = {}
err_toto = []
for i, row in df_toto_toto.iterrows():
    toto_n = row['第n回']
    toto_kind = row['種別']
    try:
        ts = get_toto_sim(toto_n, toto_kind)
        toto_dic[toto_n] = ts
    except:
        err_toto += [toto_n]
print('err : ', err_toto)

err :  []


In [217]:
toto_miniA_dic = {}
err_toto_miniA = []
for i, row in df_toto_miniA.iterrows():
    toto_n = row['第n回']
    toto_kind = row['種別']
    try:
        ts = get_toto_sim(toto_n, toto_kind)
        toto_miniA_dic[toto_n] = ts
    except:
        err_toto_miniA += [toto_n]
print('err : ', err_toto_miniA)

err :  []


In [218]:
toto_miniB_dic = {}
err_toto_miniB = []
for i, row in df_toto_miniB.iterrows():
    toto_n = row['第n回']
    toto_kind = row['種別']
    try:
        ts = get_toto_sim(toto_n, toto_kind)
        toto_miniB_dic[toto_n] = ts
    except:
        err_toto_miniB += [toto_n]
print('err : ', err_toto_miniB)

err :  []


In [219]:
df_summary_result = None
df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)
df_toto = df_toto[['第n回', '種別', '当選金_1', '当選金_2', '当選金_3', '当せん口数_1', '当せん口数_2', '当せん口数_3']].drop_duplicates()

for toto_kind in ['toto', 'mini toto-A組', 'mini toto-B組']:

    if toto_kind == 'toto':
        f_name = 'toto_dic.sav'
        price_list = [100, 200, 300, 400, 600, 800, 900, 1200, 1600, 1800, 2400, 2700, 3200, 3600, 4800, 5400, 6400, 7200, 8100, 9600, 10800, 12800, 14400, 16200, 19200, 21600, 24300, 25600, 28800, 32400, 38400, 43200, 48600]
    elif toto_kind == 'mini toto-A組':
        f_name = 'toto_miniA_dic.sav'
        price_list = [100, 200, 300, 400, 600, 800, 900, 1200, 1600, 1800, 2400, 2700, 3200, 3600, 4800, 5400, 7200]#, 8100, 10800, 16200, 24300]
    else:
        f_name = 'toto_miniB_dic.sav'
        price_list = [100, 200, 300, 400, 600, 800, 900, 1200, 1600, 1800, 2400, 2700, 3200, 3600, 4800, 5400, 7200]#, 8100, 10800, 16200, 24300]
    
    toto_dic = pickle.load(open('data/simulation_toto/' + f_name, 'rb'))
    for price in price_list:
        print('処理中 : ', toto_kind, price)
        df_result = None
        for ts in toto_dic.values():
            df = ts.get_result(price)
            df_result = pd.concat([df_result, df], axis = 0)
        df_result['種別'] = toto_kind
        df_result = df_result[df_result['くじ結果'] != '未実施']
        df_result = df_result[df_result['開催日'] > 20210000]
        df_result = df_result[df_result['開催日'] < 20220000]
        
        sum_price = 0
        toto_n_list = df_result['第n回'].drop_duplicates().values
        for n in toto_n_list:
            ts = toto_dic[n]
            _, tmp_price = ts.get_buy_list(price)
            sum_price += tmp_price
        
        tmp_toto_n = df_result[['第n回']].drop_duplicates().shape[0]
        if toto_kind == 'toto':
            miss_n = 3
        else:
            miss_n = 1
        df_result_m = df_result[df_result['外したくじ数'] < miss_n][['第n回', '種別', '開催日','外したくじ数']].drop_duplicates()
        df_marge = pd.merge(df_result_m, df_toto,how = 'left')
        win_price = 0
        for i, row in df_marge.iterrows():
            if row['外したくじ数'] == 0:
                tmp_win_price = row['当選金_1']
                if tmp_win_price == 0 and toto_kind != 'toto':
                    tmp_win_price = 14158
                    print('1等独り占め')
                elif tmp_win_price == 0 and toto_kind == 'toto':
                    tmp_win_price = 60235417
                    print('1等独り占め')
                win_price += tmp_win_price
            elif row['外したくじ数'] == 1:
                if row['当選金_2'] == 0:
                    win_price += 5732546
                    print('2等独り占め')
                else:
                    win_price += row['当選金_2']
            else:
                if row['当選金_3'] == 0:
                    win_price += 44455
                    print('3等独り占め')
                else:
                    win_price += row['当選金_3']
                
        columns = [] 
        summary_list = [] 
        columns += ['種別']
        summary_list += [toto_kind]
        columns += ['軍資金']
        summary_list += [price]
        columns += ['使用金額合計']
        summary_list += [sum_price]
        columns += ['当選金合計']
        summary_list += [win_price]
        columns += ['利益']
        summary_list += [win_price - sum_price]
        columns += ['予測したtoto数']
        summary_list += [tmp_toto_n]
        columns += ['当選数']
        summary_list += [df_marge.shape[0]]
        columns += ['1等の数']
        summary_list += [df_result_m[df_result_m['外したくじ数'] == 0].shape[0]]
        columns += ['2等の数']
        summary_list += [df_result_m[df_result_m['外したくじ数'] == 1].shape[0]]
        columns += ['3等の数']
        summary_list += [df_result_m[df_result_m['外したくじ数'] == 2].shape[0]]
        columns += ['くじ毎の平均的中試合数']
        if toto_kind == 'toto':
            n_match = 13
        else:
            n_match = 5
        summary_list += [round(n_match-df_result['外したくじ数'].mean(), 2)]
        columns += ['予測した試合数']
        a = df_result.shape[0]
        summary_list += [a]
        columns += ['的中数']
        b = df_result[df_result['当選フラグ'] == 1].shape[0]
        summary_list += [b]
        columns += ['的中率']
        summary_list += [str(round(b/a, 2)) + '%']
        columns += ['分の試合数']
        a = df_result[df_result['くじ結果'] == 0].shape[0]
        summary_list += [a]
        columns += ['分の的中数']
        b = df_result[(df_result['くじ結果'] == 0) & (df_result['当選フラグ'] == 1)].shape[0]
        summary_list += [b]
        columns += ['分の的中率']
        summary_list += [str(round(b/a, 2)) + '%']
        columns += ['勝の試合数']
        a = df_result[df_result['くじ結果'] == 1].shape[0]
        summary_list += [a]
        columns += ['勝の的中数']
        b = df_result[(df_result['くじ結果'] == 1) & (df_result['当選フラグ'] == 1)].shape[0]
        summary_list += [b]
        columns += ['勝の的中率']
        summary_list += [str(round(b/a, 2)) + '%']
        columns += ['敗の試合数']
        a = df_result[df_result['くじ結果'] == 2].shape[0]
        summary_list += [a]
        columns += ['敗の的中数']
        b = df_result[(df_result['くじ結果'] == 2) & (df_result['当選フラグ'] == 1)].shape[0]
        summary_list += [b]
        columns += ['敗の的中率']
        summary_list += [str(round(b/a, 2)) + '%']
        
        df_tmp = pd.DataFrame([summary_list], columns = columns)
        df_summary_result = pd.concat([df_summary_result, df_tmp])

処理中 :  toto 100
処理中 :  toto 200
処理中 :  toto 300
処理中 :  toto 400
処理中 :  toto 600
処理中 :  toto 800
処理中 :  toto 900
処理中 :  toto 1200
処理中 :  toto 1600
処理中 :  toto 1800
処理中 :  toto 2400
処理中 :  toto 2700
処理中 :  toto 3200
処理中 :  toto 3600
処理中 :  toto 4800
処理中 :  toto 5400
処理中 :  toto 6400
処理中 :  toto 7200
処理中 :  toto 8100
処理中 :  toto 9600
処理中 :  toto 10800
処理中 :  toto 12800
処理中 :  toto 14400
処理中 :  toto 16200
処理中 :  toto 19200
処理中 :  toto 21600
処理中 :  toto 24300
処理中 :  toto 25600
処理中 :  toto 28800
処理中 :  toto 32400
処理中 :  toto 38400
処理中 :  toto 43200
処理中 :  toto 48600
処理中 :  mini toto-A組 100
処理中 :  mini toto-A組 200
処理中 :  mini toto-A組 300
処理中 :  mini toto-A組 400
処理中 :  mini toto-A組 600
処理中 :  mini toto-A組 800
処理中 :  mini toto-A組 900
処理中 :  mini toto-A組 1200
処理中 :  mini toto-A組 1600
処理中 :  mini toto-A組 1800
処理中 :  mini toto-A組 2400
処理中 :  mini toto-A組 2700
処理中 :  mini toto-A組 3200
処理中 :  mini toto-A組 3600
処理中 :  mini toto-A組 4800
処理中 :  mini toto-A組 5400
処理中 :  mini toto-A組 7200
処理中 :  mini tot

In [220]:
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 150)

In [221]:
df_summary_result[df_summary_result['利益'] > 0].sort_values('利益')

,種別,軍資金,使用金額合計,当選金合計,利益,予測したtoto数,当選数,1等の数,2等の数,3等の数,くじ毎の平均的中試合数,予測した試合数,的中数,的中率,分の試合数,分の的中数,分の的中率,勝の試合数,勝の的中数,勝の的中率,敗の試合数,敗の的中数,敗の的中率


In [222]:
df_summary_result[df_summary_result['軍資金'] == 100].sort_values('利益')

,種別,軍資金,使用金額合計,当選金合計,利益,予測したtoto数,当選数,1等の数,2等の数,3等の数,くじ毎の平均的中試合数,予測した試合数,的中数,的中率,分の試合数,分の的中数,分の的中率,勝の試合数,勝の的中数,勝の的中率,敗の試合数,敗の的中数,敗の的中率
0,mini toto-A組,100,1600,0,-1600,16,0,0,0,0,1.81,80,29,0.36%,26,0,0.0%,23,16,0.7%,28,10,0.36%
0,toto,100,1300,0,-1300,13,0,0,0,0,4.77,169,62,0.37%,47,4,0.09%,65,41,0.63%,56,16,0.29%
0,mini toto-B組,100,1600,350,-1250,16,1,1,0,0,2.50,80,40,0.5%,15,0,0.0%,42,28,0.67%,21,10,0.48%


In [ ]:
1205600/38